In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\RafaelRaiserQSOFT\\Stellar-Classification'

In [2]:
import pandas as pd

C:\Users\RafaelRaiserQSOFT\AppData\Local\Temp\ipykernel_3964\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
cols2read=['alpha','delta','u','g','r','i','z','redshift','obj_ID','class']

data_raw=pd.read_csv('artifacts/data_ingestion/data.csv',usecols=cols2read,dtype={'obj_ID':str})
data_raw.set_index('obj_ID',inplace=True)
data_raw.head()

,alpha,delta,u,g,r,i,z,class,redshift
obj_ID,,,,,,,,,
1237660961327743232,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,GALAXY,0.634794
1237664879951151360,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,GALAXY,0.779136
1237660961330430208,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,GALAXY,0.644195
1237663478724297984,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,GALAXY,0.932346
1237680272041378048,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,GALAXY,0.116123


In [4]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 1237660961327743232 to 1237661151378801152
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   alpha     100000 non-null  float64
 1   delta     100000 non-null  float64
 2   u         100000 non-null  float64
 3   g         100000 non-null  float64
 4   r         100000 non-null  float64
 5   i         100000 non-null  float64
 6   z         100000 non-null  float64
 7   class     100000 non-null  object 
 8   redshift  100000 non-null  float64
dtypes: float64(8), object(1)
memory usage: 7.6+ MB


In [5]:
data_raw.isna().sum()

alpha       0
delta       0
u           0
g           0
r           0
i           0
z           0
class       0
redshift    0
dtype: int64

In [6]:
data_raw.shape

(100000, 9)

In [7]:
data_raw.columns

Index(['alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'class', 'redshift'], dtype='object')

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    local_data_file: Path
    all_schema: dict

In [9]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            local_data_file = config.local_data_file,
            all_schema=schema,
        )

        return data_validation_config

In [11]:
import os
from src import logger

In [12]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.local_data_file)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    print(col)
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [13]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-02-15 11:20:10,049: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-15 11:20:10,051: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-15 11:20:10,054: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-15 11:20:10,056: INFO: common: created directory at: artifacts]
[2024-02-15 11:20:10,057: INFO: common: created directory at: artifacts/data_validation]
